##  Graph 对象

In [1]:
import tensorflow as tf

In [3]:
g1 = tf.Graph()
g2 = tf.Graph()

In [4]:
with g1.as_default():
    # 定义g1中的Op张量等
    pass

with g2.as_default():
    # 定义g2中的Op张量等
    pass


## Session对象
1. 参数
 + target：指定使用的执行引擎
 + graph ：指定将要在Session中加载的Graph对象，默认为None，指使用默认的Graph对象，当有多个数据流程图的时候，最好传入希望运行的Graph对象
 + config：指定配置Session所需的选项

2. Session.run()
 + fetches 参数，希望执行的Op或者是Tensor对象
 + feed_dict 参数，用于覆盖数据流图中的Tensor对象值
 ```python
 import tensorflow as tf
 # 创建Op， Tensor对象等
 a = tf.add(2, 5)
 b = tf.add(a, 3)
 # 利用默认的数据流图创建Session对象
 sess = tf.Session()
 # 定义一个字典，比如将a的值替换为15
 replace_dict = {a:15}
 # 运行Session对象(结果将为18)
 sess.run(b, feed_dict = replace_dict)
 ```
3. Session调用完成自后要调用close方法，释放资源
 ```python
 sess = tf.Session()
 ....
 sess.close()
 ```
 或者使用下面的方法达到自动关闭的目的
 ```python
 with tf.Session() as sess:
     # 运行数据流图，写入概括统计量等
     ....
 # Session对象自动关闭
 ```


可以使用Session.as_default() 将Session作为上下文管理器使用，
如下面的a.eval()就相当于sess.run(a).但是以这种方式来使用session的时候就必须手动关闭Session
如果没有Session作为上下文环境的话，单独使用object.eval()的时候已经会出现错误

In [9]:
a = tf.constant(5)
b = tf.constant(3)
c = tf.add(a, b)
sess = tf.Session()
with sess.as_default():
    print(a.eval())
    print(b.eval())
    print(c.eval())

# 必须手动关闭Session对象
sess.close()

5
3
8


## 利用占位符进行输入
通过上面的例子可以看到，我们的输入节点都是使用的一些固定的值，但是我们在处理数据流程图的时候往往是要动态进行数据的输入
那么，我们怎么来进行数据的动态输入呢？下面就是利用占位符进行数据的站位输入

In [15]:
import numpy as np
# 创建一个长度为2， 数据类型为int32的占位向量
a = tf.placeholder(tf.int32, shape = [2], name = 'my_input')

# 将占位向量视为其他任意的Tensor对象，加以使用
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")

# 完成数据流图的定义
d = tf.add(b, c, name="add_d")

对上面的内容的解释：
1. tf.placeholder(dtype, shape, name)这个函数用来创建占位符。
2. 其中dtype参数必须有值，按照tensorflow的介绍，他里面提供的参数类型值应该和numpy中提供的相同，但最好还是使用它内部提供的类型，提供一致性
3. shape可有可无，没有的话就是默认可以接受任何形状的数据

下面我们给这个占位符一个输入，然后运行这个数据流图

In [25]:
input_dict = {a:np.array([4, 67] ,dtype = np.int32)}
with tf.Session() as sess :
    print(sess.run(d, feed_dict = input_dict))

339


对上面的解释：
首先我们创建了一个字典，里面的key是我们的Tensor的占位符的句柄
它的value是我们的输入值，这里我们创建了一个shape为[2]的np.array()
从创建的语句我们其实可以看得出来，tensorflow中的tf.int32 和numpy中的np.int32其实是等价的
有了输入，我们就可以运行这个数据流图了

另外，如果我们没有给这个数据流图中的a赋值，那么是无法运行节点d的，因为d依赖于a。a没有值，则d无法计算

## Variable对象
Tensor对象和Op对象都是不可变的，但是机器学习的本质决定了我们需要一种机制来保存随时间变化的值（这一点从深度学习的模型中就可以观察的出来），借助TensorFlow中的Variable对象可以达到这个目的

In [27]:
# 创建一个初始值为5的variable对象
my_var = tf.Variable(5, name="my_variable")
add = tf.add(5, my_var)
mul = tf.multiply(3, my_var)

通常variable对象的初始值是全0或者全1或者用随机数填充的阶数较高的张量
为此，TensorFlow提供了一些辅助的Op来创建我们需要的张量

In [36]:
# 2 × 2的矩阵
zeros = tf.zeros([2,2])
print("zeros:" + str(zeros))

# 长度为6的全1矩阵
ones = tf.ones([6])
print("ones:" + str(ones))

# 3 × 3 × 3 的张量，其元素服从0~10的均匀分布
uniform = tf.random_uniform([3, 3, 3], minval=0, maxval=10)
print("uniform[0~10]:" + str(uniform))

# 3 × 3 × 3 的张量，其元素服从0均值，标准差为2的正态分布
normal = tf.random_normal([3,3,3], mean=0.0, stddev=2.0)
print("normal[0, 2]:" + str(normal))

# 下面创建的这个张量不会返回超过2倍标准差的值，这样可以防止有一个或两个元素与该张量中其他元素有显著不同
# 下面创建的这个张量不会返回任何大于7或者小于3的值
trunc = tf.truncated_normal([2,2], mean=5.0, stddev=1.0)
print("trunc_3<x<7:"+ str(trunc))

zeros:Tensor("zeros_5:0", shape=(2, 2), dtype=float32)
ones:Tensor("ones_4:0", shape=(6,), dtype=float32)
uniform[0~10]:Tensor("random_uniform_3:0", shape=(3, 3, 3), dtype=float32)
normal[0, 2]:Tensor("random_normal_2:0", shape=(3, 3, 3), dtype=float32)
trunc_3<x<7:Tensor("truncated_normal:0", shape=(2, 2), dtype=float32)


#### Variable对象初始化